### Difference Features

In [23]:
# !sudo apt update
# !sudo apt install openjdk-17-jre-headless -y
import pyspark
from pyspark import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import col, substring
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType


In [2]:
conf = pyspark.SparkConf().setAll([\
            ('spark.app.name', 'Glucose_Analysis_Spark')])\
            .set('spark.sql.shuffle.partitions', '1500')
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()  

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 23:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.options(header='True', inferSchema='True', delimiter=',')\
        .csv('/cephfs/data/cohort.csv')

In [4]:
df.show(2)

23/05/02 23:28:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+---+--------------------+------+-------------------+---+------------+---------+
|_c0|              UserId|Gender|                DOB|Age|DiabetesType|Treatment|
+---+--------------------+------+-------------------+---+------------+---------+
|  0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|
|  1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|
+---+--------------------+------+-------------------+---+------------+---------+
only showing top 2 rows



In [27]:
df = df.withColumn('AgeGroup', substring(df.Age.cast(StringType()), 0,1))

In [30]:
df = df.withColumnRenamed('Gender', 'Sex')

In [31]:
df.show(3)

23/05/02 23:42:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+---+--------------------+------+-------------------+---+------------+---------+--------+
|_c0|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|
+---+--------------------+------+-------------------+---+------------+---------+--------+
|  0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|       9|
|  1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|       8|
|  2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|       8|
+---+--------------------+------+-------------------+---+------------+---------+--------+
only showing top 3 rows



In [32]:
encodedCols = ['Sex', 'Treatment', 'AgeGroup'] # not doing'DiabetesType' because all type-two
encodedLabels = []

for name in encodedCols:
    indexer = StringIndexer(inputCol=name, outputCol= name + '_Num')
    indexer_fitted = indexer.fit(df)
    encodedLabels.append([name, indexer_fitted.labels])
                          
    df = indexer_fitted.transform(df)

In [33]:
encodedLabels

[['Sex', ['Female', 'Male']],
 ['Treatment', ['yes-both', 'yes-long-acting', 'no', 'yes-fast-acting']],
 ['AgeGroup', ['5', '6', '7', '4', '3', '8', '9', '1']]]

In [34]:
df.show(2)

23/05/02 23:42:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+
|_c0|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|Sex_Num|Treatment_Num|AgeGroup_Num|
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+
|  0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|       9|    0.0|          2.0|         6.0|
|  1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|       8|    1.0|          2.0|         5.0|
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+--

In [35]:
single_col_ohe = OneHotEncoder(inputCol="Sex_Num", outputCol="Sex_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

single_col_ohe = OneHotEncoder(inputCol="Treatment_Num", outputCol="Treatment_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

single_col_ohe = OneHotEncoder(inputCol="AgeGroup_Num", outputCol="AgeGroup_Encoded", dropLast=True)
df = single_col_ohe.fit(df).transform(df)

In [36]:
df.show()

23/05/02 23:43:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+-------------+-----------------+----------------+
|_c0|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|Sex_Num|Treatment_Num|AgeGroup_Num|  Sex_Encoded|Treatment_Encoded|AgeGroup_Encoded|
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+-------------+-----------------+----------------+
|  0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|       9|    0.0|          2.0|         6.0|(1,[0],[1.0])|    (3,[2],[1.0])|   (7,[6],[1.0])|
|  1|9qY9mZ+GV5Kd/O/NB...|  

In [38]:
df.write.parquet('/cephfs/data/cohort_encoded.parquet')

23/05/02 23:44:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv


In [39]:
test = spark.read.parquet('/cephfs/data/cohort_encoded.parquet')

In [40]:
test.show(4)

+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+-------------+-----------------+----------------+
|_c0|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|Sex_Num|Treatment_Num|AgeGroup_Num|  Sex_Encoded|Treatment_Encoded|AgeGroup_Encoded|
+---+--------------------+------+-------------------+---+------------+---------+--------+-------+-------------+------------+-------------+-----------------+----------------+
|  0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|       9|    0.0|          2.0|         6.0|(1,[0],[1.0])|    (3,[2],[1.0])|   (7,[6],[1.0])|
|  1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|       8|    1.0|          2.0|         5.0|    (1,[],[])|    (3,[2],[1.0])|   (7,[5],[1.0])|
|  2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|       8|    0.0|          2.0|         5.0|(1,[0]